In [34]:
# basic dependencies

import numpy as np
from numpy import loadtxt
from numpy import savetxt

import pandas as pd
import math
import time
from datetime import date
from pathlib import Path
import os

np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})

###########

# torch dependencies
import torch

tkwargs = {"dtype": torch.double, # set as double to minimize zero error for cholesky decomposition error
           #"device": torch.device("cuda:0" if torch.cuda.is_available() else "cpu")} # set tensors to GPU, if multiple GPUs please set cuda:x properly
           "device": torch.device("cpu")}

torch.set_printoptions(precision=3)

###########

# botorch dependencies
import botorch

# data related
from botorch.utils.sampling import draw_sobol_samples
from botorch.utils.transforms import unnormalize, normalize

# surrogate model specific
from botorch.models.gp_regression import SingleTaskGP, FixedNoiseGP
from botorch.models.model_list_gp_regression import ModelListGP
from botorch.models.transforms.outcome import Standardize
from gpytorch.mlls.sum_marginal_log_likelihood import SumMarginalLogLikelihood
from botorch import fit_gpytorch_model

# qNEHVI specific
from botorch.acquisition.multi_objective.objective import IdentityMCMultiOutputObjective
from botorch.acquisition.multi_objective.monte_carlo import qNoisyExpectedHypervolumeImprovement

# utilities
from botorch.optim.optimize import optimize_acqf, optimize_acqf_list
from botorch.sampling.samplers import SobolQMCNormalSampler
from botorch.utils.multi_objective.pareto import is_non_dominated
from botorch.utils.multi_objective.hypervolume import Hypervolume
from typing import Optional
from torch import Tensor
from botorch.exceptions import BadInitialCandidatesWarning

import warnings

warnings.filterwarnings('ignore', category=BadInitialCandidatesWarning)
warnings.filterwarnings('ignore', category=RuntimeWarning)

###########

# pymoo dependencies
import pymoo

from pymoo.factory import get_problem
from pymoo.core.problem import ElementwiseProblem

from pymoo.algorithms.moo.nsga3 import NSGA3
from pymoo.algorithms.moo.unsga3 import UNSGA3
from pymoo.util.ref_dirs import get_reference_directions
from pymoo.optimize import minimize
from pymoo.core.termination import NoTermination

from pymoo.core.problem import Problem

from sympy.utilities.iterables import multiset_permutations


In [5]:
########## settings to change every day

total_exp = 30 # total sets of experiments to run today

algo = 'pure' # start from pure
BATCH_SIZE = 4

########## settings to keep, used for some parts of the code

exp_counter = 5 # current number of experiments done, start with 0 for initialization

n_var = 5
n_obj = 3
n_constr = 2

random_state = 1
torch.manual_seed(random_state) # gives a consistent seed based on the trial number

ref_point = torch.tensor([0, 0, 0], **tkwargs)
hv=Hypervolume(ref_point=ref_point)

initial_sample_size = 12 # no of initialized LHS samples

In [3]:
while exp_counter < total_exp+1:
    t0 = time.time()
     
    ##########
    # load data from matlab/labview into jupyter       
    if algo == 'pure':
        input_file = "Data Analysis/Algo1/Results_Algo1_Run" + str(exp_counter) + ".csv"
        if Path(input_file).is_file() == False:
            print("No input file detected, sleeping for 30 sec")
            time.sleep(30)
            continue;
            
        initial_x_pandas = pd.read_csv(f"{input_file}", delimiter=',') 
        initial_x_torch = torch.tensor(initial_x_pandas.iloc[:,1:].values, **tkwargs) 
        
    else:
        input_file = "Data Analysis/Algo2/Results_Algo2_Run" + str(exp_counter) + ".csv"
        if Path(input_file).is_file() == False:
            print("No input file detected, sleeping for 30 sec")
            time.sleep(30)
            continue;
            
        initial_x_pandas = pd.read_csv(f"{input_file}", delimiter=',') 
        initial_x_torch = torch.tensor(initial_x_pandas.iloc[:,1:].values, **tkwargs) 
        
    print("File successfully found! Proceeding with optimization.")

    ####################
    # initial training data
    train_x = initial_x_torch[:,:n_var]
    train_obj = initial_x_torch[:,n_var:n_var+n_obj]

    ####################
    # normalization

    # normalize inputs to [0,1] first before feeding into model
    problem_bounds = torch.zeros(2, n_var, **tkwargs)
    problem_bounds[0] = 0.6
    problem_bounds[1] = 24.0

    standard_bounds = torch.zeros(2, n_var, **tkwargs)
    standard_bounds[1] = 1

    ####################
    # surrogate model
    
    # define and train surrogate models for objective and constraint
    models = []
    
    # models for objective
    train_x_gp = normalize(train_x, problem_bounds)
    for i in range(train_obj.shape[-1]):
        models.append(SingleTaskGP(train_x_gp, train_obj[..., i : i + 1], outcome_transform=Standardize(m=1)))

    model = ModelListGP(*models)
    mll = SumMarginalLogLikelihood(model.likelihood, model)

    fit_gpytorch_model(mll) 
    
    acq_func = qNoisyExpectedHypervolumeImprovement(
        model=model,
        ref_point=ref_point, # for computing HV, must flip for BoTorch
        X_baseline=train_x_gp, # feed total list of train_x for this current iteration
        sampler=SobolQMCNormalSampler(num_samples=128),  # determines how candidates are randomly proposed before selection
        objective=IdentityMCMultiOutputObjective(outcomes=np.arange(n_obj).tolist()), # optimize first n_obj col 
        #constraints=create_idxrs(), # constraint on last n_constr col
        prune_baseline=True, cache_pending=True)  # options for improving qNEHVI, keep these on

    ####################    
    # optimization

    if algo == 'pure':

        # taking 256 x 2 QMC candidates
        sobol_x1 = draw_sobol_samples(bounds=problem_bounds,n=256, q=1).squeeze(1)
        sobol_x2 = draw_sobol_samples(bounds=problem_bounds,n=256, q=1).squeeze(1)
        sobol_x = torch.cat([sobol_x1, sobol_x2])

        # repair inputs to feasibility
        for i in range(sobol_x.shape[0]):
            if 0.3 - sobol_x[i,1]/sobol_x[i,4] > 0:
                sobol_x[i,4] = min(sobol_x[i,1]/0.3, sobol_x[i,4])

        for i in range(sobol_x.shape[0]):
            if 2 - (sobol_x[i,1]/sobol_x[i,4]) - (sobol_x[i,3]/sobol_x[i,1]) > 0:
                sobol_x[i,3] = max((2-sobol_x[i,1]/sobol_x[i,4])*sobol_x[i,1], sobol_x[i,3])

        sobol_x_gp = normalize(sobol_x, problem_bounds)

        acq_value_list = []
        for i in range(0, sobol_x_gp.shape[0]):
            with torch.no_grad():
                acq_value = acq_func(sobol_x_gp[i].unsqueeze(dim=0))
                acq_value_list.append(acq_value.item())

        # sorted
        sorted_x = sobol_x_gp.cpu().numpy()[np.argsort((acq_value_list))]

        # take best BATCH_SIZE samples from sorted pool
        new_x = torch.tensor(sorted_x[-BATCH_SIZE:], **tkwargs)  

    else:
        
        ####################
        # BO half
        # taking 256 x 1 QMC candidates
        sobol_x = draw_sobol_samples(bounds=problem_bounds,n=256, q=1).squeeze(1)

        # repair inputs to feasibility
        for i in range(sobol_x.shape[0]):
            if 0.3 - sobol_x[i,1]/sobol_x[i,4] > 0:
                sobol_x[i,4] = min(sobol_x[i,1]/0.3, sobol_x[i,4])

        for i in range(sobol_x.shape[0]):
            if 2 - (sobol_x[i,1]/sobol_x[i,4]) - (sobol_x[i,3]/sobol_x[i,1]) > 0:
                sobol_x[i,3] = max((2-sobol_x[i,1]/sobol_x[i,4])*sobol_x[i,1], sobol_x[i,3])

        sobol_x_gp = normalize(sobol_x, problem_bounds)

        acq_value_list = []
        for i in range(0, sobol_x_gp.shape[0]):
            with torch.no_grad():
                acq_value = acq_func(sobol_x_gp[i].unsqueeze(dim=0))
                acq_value_list.append(acq_value.item())

        # filter the best 4 QMC candidates first
        sorted_x = sobol_x_gp.cpu().numpy()[np.argsort((acq_value_list))]
        qnehvi_x = torch.tensor(sorted_x[-4:], **tkwargs)  

        ####################
        # EA half
        # we pick out the best points so far to form parents
        pareto_mask = is_non_dominated(train_obj)
        pareto_y = -train_obj[pareto_mask]
        pareto_x = train_x_gp[pareto_mask]

        algorithm = UNSGA3(pop_size=256,
                           ref_dirs=get_reference_directions("energy", n_obj, BATCH_SIZE, seed=random_state),
                           sampling=pareto_x.cpu().numpy(),
                          )

        pymooproblem = Problem(n_var=n_var, n_obj=n_obj, xl=np.zeros(n_var), xu=np.ones(n_var))
        algorithm.setup(pymooproblem, termination=NoTermination())

        # set the 1st population to the current evaluated population
        pop = algorithm.ask()
        pop.set("F", pareto_y.cpu().numpy())
        algorithm.tell(infills=pop)

        # propose children based on tournament selection -> crossover/mutation
        newpop = algorithm.ask()
        nsga3_x = torch.tensor(newpop.get("X"), **tkwargs)
        nsga3_x = unnormalize(nsga3_x, bounds=problem_bounds)

        # repair inputs to feasibility
        for i in range(nsga3_x.shape[0]):
            if 0.3 - nsga3_x[i,1]/nsga3_x[i,4] > 0:
                nsga3_x[i,4] = min(nsga3_x[i,1]/0.3, nsga3_x[i,4])

        for i in range(nsga3_x.shape[0]):
            if 2 - (nsga3_x[i,1]/nsga3_x[i,4]) - (nsga3_x[i,3]/nsga3_x[i,1]) > 0:
                nsga3_x[i,3] = max((2-nsga3_x[i,1]/nsga3_x[i,4])*nsga3_x[i,1], nsga3_x[i,3])
                
        nsga3_x = normalize(nsga3_x, problem_bounds)

        ####################
        # combine BO and EA        
        # total pool of candidates for sorting
        candidates = torch.cat([qnehvi_x, nsga3_x])

        acq_value_list = []
        for i in range(0, candidates.shape[0]):
            with torch.no_grad():
                acq_value = acq_func(candidates[i].unsqueeze(dim=0))
                acq_value_list.append(acq_value.item())

        # take best BATCH_SIZE samples from sorted pool
        sorted_x = candidates.cpu().numpy()[np.argsort((acq_value_list))]
        new_x = torch.tensor(sorted_x[-BATCH_SIZE:], **tkwargs)    

    #################### 
    # normalization and repair

    # unormalize our training inputs back to original problem bounds
    new_x =  unnormalize(new_x, bounds=problem_bounds)
    
    # create a list to indicate if repair was done
    repair_array = np.zeros((new_x.shape[0], 1))
    
    constraint_array1 = np.zeros((new_x.shape[0], 1))
    constraint_array2 = np.zeros((new_x.shape[0], 1))
    
    # record prior constraint values
    for i in range(new_x.shape[0]):
        constraint_array1[i] = 0.3 - new_x[i,1]/new_x[i,4]
        constraint_array2[i] = 2 - (new_x[i,1]/new_x[i,4]) - (new_x[i,3]/new_x[i,1])


    #################### 
    t1 = time.time()
    
    # convert back to matlab/labview format for downloading
    new_x_pandas = pd.DataFrame(new_x.cpu().numpy(), columns=['Qtsc', 'Qag', 'Qpva', 'Qseed', 'Qaa'])
    new_x_pandas['Cond'] = new_x_pandas.index + BATCH_SIZE*exp_counter + initial_sample_size + 1
    new_x_pandas['Repair'] = repair_array
    new_x_pandas['Violation1'] = constraint_array1
    new_x_pandas['Violation2'] = constraint_array2
    new_x_pandas['Time'] =  t1-t0
    new_x_pandas = new_x_pandas[['Cond', 'Qtsc', 'Qag', 'Qpva', 'Qseed', 'Qaa', 'Repair', 'Violation1', 'Violation2', 'Time']] 
        
        
    # Save new flowrate values in csv files before shuffling
    if algo == 'pure':
        output_file = "Flowrates/Flowrates_Algo1_BS_Run" + str(exp_counter+1)
        new_x_pandas.to_csv(f'{output_file}.csv', index=False)
        
    else:
        output_file = "Flowrates/Flowrates_Algo2_BS_Run" + str(exp_counter+1)
        new_x_pandas.to_csv(f'{output_file}.csv', index=False)
        
         # Shuffle flowrate
        input_file = "Flowrates/Flowrates_Algo1_BS_Run" + str(exp_counter+1) + ".csv"            
        dataAlgo1_pandas = pd.read_csv(f"{input_file}", delimiter=',') 
        dataAlgo1_torch = torch.tensor(dataAlgo1_pandas.iloc[:,:].values, **tkwargs)

        input_file = "Flowrates/Flowrates_Algo2_BS_Run" + str(exp_counter+1) + ".csv"            
        dataAlgo2_pandas = pd.read_csv(f"{input_file}", delimiter=',') 
        dataAlgo2_torch = torch.tensor(dataAlgo2_pandas.iloc[:,:].values, **tkwargs)
   
        iOrder = [0,4,1,5,2,6,3,7]
        iSave = 4
        pSave = [0,1,2,3]
        qSave = [0,1,2,3]
            
        for p in multiset_permutations([0,1,2,3]):
            for q in multiset_permutations([0,1,2,3]):
                dataAlgo_torch = torch.vstack([dataAlgo1_torch[p,1:6], dataAlgo2_torch[q,1:6]])
                dataAlgoOrder_torch = dataAlgo_torch[iOrder]

                Test = dataAlgoOrder_torch
                Test[Test <= 4] = 1
                Test[Test > 4] = 0

                # Cumulative sum
                cumsumsave = torch.zeros(2*BATCH_SIZE-1, 1, **tkwargs)
                for i in range(2*BATCH_SIZE-1):
                    cumsum = torch.sum(Test[i:i+2],0)
                    cumsumsave[i] = max(cumsum)

                if(max(cumsumsave) < 2) & (iSave > 1):
                    pSave = p
                    qSave = q
                    iSave = 1
                    print(Test)
                    print(iSave)
                    print(pSave)
                    print(qSave)
                else:
                    # Cumulative sum
                    cumsumsave = torch.zeros(2*BATCH_SIZE-2, 1, **tkwargs)
                    for i in range(2*BATCH_SIZE-2):
                        cumsum = torch.sum(Test[i:i+3],0)
                        cumsumsave[i] = max(cumsum)
                    
                    if(max(cumsumsave) < 3) & (iSave > 2):
                        pSave = p
                        qSave = q
                        iSave = 2
                        print(Test)
                        print(iSave)
                        print(pSave)
                        print(qSave)
                    else:
                        # Cumulative sum
                        cumsumsave = torch.zeros(2*BATCH_SIZE-3, 1, **tkwargs)
                        for i in range(2*BATCH_SIZE-3):
                            cumsum = torch.sum(Test[i:i+4],0)
                            cumsumsave[i] = max(cumsum)

                        if(max(cumsumsave) < 4) & (iSave > 3):
                            pSave = p
                            qSave = q
                            iSave = 3
                            print(Test)
                            print(iSave)
                            print(pSave)
                            print(qSave)
                
        print(pSave)
        print(qSave)
        
        dataAlgo_torch = torch.vstack([dataAlgo1_torch[pSave], dataAlgo2_torch[qSave]])
        dataAlgoOrder_torch = dataAlgo_torch[iOrder]
        new_dataAlgo1_pandas = pd.DataFrame(dataAlgo1_torch[pSave,:].cpu().numpy(), columns=['Cond', 'Qtsc', 'Qag', 'Qpva', 'Qseed', 'Qaa', 'Repair', 'Violation1', 'Violation2', 'Time'])
        new_dataAlgo2_pandas = pd.DataFrame(dataAlgo2_torch[qSave,:].cpu().numpy(), columns=['Cond', 'Qtsc', 'Qag', 'Qpva', 'Qseed', 'Qaa', 'Repair', 'Violation1', 'Violation2', 'Time'])
        
        new_dataAlgo1_pandas['Cond'] = new_dataAlgo1_pandas.index + BATCH_SIZE*exp_counter + initial_sample_size + 1
        new_dataAlgo2_pandas['Cond'] = new_dataAlgo2_pandas.index + BATCH_SIZE*exp_counter + initial_sample_size + 1


        # Save new flowrate values in csv files after shuffling
        output_file = "Flowrates/Flowrates_Algo1_Run" + str(exp_counter+1)
        new_dataAlgo1_pandas.to_csv(f'{output_file}.csv', index=False)

        output_file = "Flowrates/Flowrates_Algo2_Run" + str(exp_counter+1)
        new_dataAlgo2_pandas.to_csv(f'{output_file}.csv', index=False)

        
    #################### 
    # end of loop
        
    if algo == 'pure':
        print(f"Experiment number {exp_counter} for pure BO , time taken: {t1-t0:>4.2f}s.\nTotal experiments run: {exp_counter}")
        algo = 'hybrid' # switch to next algo

    else:
        print(f"Experiment number {exp_counter} for hybrid BO, time taken: {t1-t0:>4.2f}s.\nTotal experiments run: {exp_counter}")
        algo = 'pure'  # switch to next algo
        exp_counter+=1
    
    print("Sleeping for 5 seconds before next experiment.")
    time.sleep(5) # pause for x sec
    

File successfully found! Proceeding with optimization.


C:\ProgramData\Anaconda3\lib\site-packages\gpytorch\lazy\lazy_tensor.py:1811: UserWarning: torch.triangular_solve is deprecated in favor of torch.linalg.solve_triangularand will be removed in a future PyTorch release.
torch.linalg.solve_triangular has its arguments reversed and does not return a copy of one of the inputs.
X = torch.triangular_solve(B, A).solution
should be replaced with
X = torch.linalg.solve_triangular(A, B). (Triggered internally at  C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\BatchLinearAlgebra.cpp:2189.)
  Linv = torch.triangular_solve(Eye, L, upper=False).solution


Experiment number 5 for pure BO , time taken: 11.04s.
Total experiments run: 5
Sleeping for 5 seconds before next experiment.
File successfully found! Proceeding with optimization.


C:\Users\User\AppData\Local\Temp\ipykernel_1732\3043205304.py:136: DeprecationWarning: Call to deprecated function (or staticmethod) get_reference_directions. (Please use `from pymoo.util.ref_dirs import get_reference_directions`)
  ref_dirs=get_reference_directions("energy", n_obj, BATCH_SIZE, seed=random_state),


tensor([[1., 0., 0., 0., 0.],
        [0., 0., 1., 1., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 1., 1., 0.],
        [1., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 0.]], dtype=torch.float64)
2
[0, 1, 2, 3]
[0, 1, 2, 3]
tensor([[1., 0., 0., 0., 0.],
        [0., 0., 1., 1., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 1., 1., 0.],
        [1., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0.],
        [1., 0., 0., 0., 0.]], dtype=torch.float64)
1
[0, 1, 2, 3]
[0, 1, 3, 2]
[0, 1, 2, 3]
[0, 1, 3, 2]
Experiment number 5 for hybrid BO, time taken: 16.96s.
Total experiments run: 5
Sleeping for 5 seconds before next experiment.
No input file detected, sleeping for 30 sec
No input file detected, sleeping for 30 sec
No input file detected, sleeping for 30 sec
No input file detected, sleeping for 30 sec
No input file detected, sleeping for 30 sec
No input file detected, sleeping for 30 sec
No 

KeyboardInterrupt: 

In [55]:
0.3 - new_x[:,1]/new_x[:,4]

tensor([-0.899, -1.324, -1.005, -0.993], dtype=torch.float64)

In [56]:
2 - (new_x[:,1]/new_x[:,4]) - (new_x[:,3]/new_x[:,1])

tensor([ 0.000e+00, -5.551e-17,  0.000e+00, -1.459e-02], dtype=torch.float64)